In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
 !pip install --upgrade pip
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import seaborn as sns
import numpy as np
sns.set(style="white")
sns.set(style="whitegrid", color_codes=True)
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score
from sklearn.metrics import accuracy_score
! pip install -q scikit-plot
import scikitplot as skplt

df = pd.read_csv('/kaggle/input/drug-addiction-in-bangladesh-reasons-smote/DrugAddictioninBangladesh_Smoted.csv')

df.head(100)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split, cross_val_score,KFold,StratifiedKFold, GridSearchCV
from sklearn.metrics import confusion_matrix,accuracy_score,f1_score,average_precision_score,recall_score,roc_auc_score
from sklearn.preprocessing import RobustScaler,StandardScaler,LabelEncoder,MinMaxScaler

from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.feature_selection import SelectKBest,chi2

from keras.models import Sequential
from keras.layers import Activation,BatchNormalization
from keras.layers.core import Dense,Dropout
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.callbacks import ReduceLROnPlateau,EarlyStopping

In [ ]:
data = pd.read_csv('/kaggle/input/drug-addiction-in-bangladesh-reasons-smote/DrugAddictioninBangladesh_Smoted.csv')
data.head()

In [ ]:
data.duplicated().sum()

In [ ]:
data = data.drop_duplicates()

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data.isna().sum()

In [ ]:
num_columns = data.shape[1]

In [ ]:
corr = data.corr()

In [ ]:
f,ax = plt.subplots(figsize=(22, 22))
sns.heatmap(corr, annot=True, linewidths=.5, fmt= '.1f',ax=ax)

In [ ]:
sns.countplot(x='Class',data=data)

In [ ]:
print(data.shape)

In [ ]:
data_norm = data.copy()

In [ ]:
colunas_normalizar = ['Education','Spend most time','Mental/emotional problem','Family relationship','Financials of family','Addicted person in family','Living with drug user','Ever taken drug','Friends influence','If chance given to taste drugs']

tipo_scaler = 'MinMax'
if(tipo_scaler=='Standard'):
    scaler = StandardScaler((0,1))
elif(tipo_scaler=='Robust'):
    scaler = RobustScaler()
elif(tipo_scaler=='MinMax'):
    scaler = MinMaxScaler(feature_range=(0, 1))

for col in colunas_normalizar:
    data_norm[col] = scaler.fit_transform(data_norm[col].values.reshape(-1,1))

In [ ]:
data_norm.head()

In [ ]:
from sklearn import preprocessing
X = data_norm.drop('Class',axis=1).values
Y = data_norm['Class'].values

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.20,random_state=0)

In [ ]:
logreg = LogisticRegression(C=1,max_iter=10000,penalty='l1',solver='liblinear')
logreg.fit(X_train,y_train)

In [ ]:
#List to compute metrics
accuracy = []
precision =[]
recall = []
f1 = []
roc = []

In [ ]:
#Logistic Regression
logreg = LogisticRegression(C=1,max_iter=10000,penalty='l1',solver='liblinear')
logreg.fit(X_train,y_train)

In [ ]:
log_reg_score = cross_val_score(logreg, X_train, y_train, cv=10,scoring='roc_auc_ovo')
log_reg_score_teste = cross_val_score(logreg, X_test, y_test, cv=10,scoring='roc_auc_ovo')
print('Score Logistic Regression Training: ', round(log_reg_score.mean() * 100, 2).astype(str) + '%')
print('Score Logistic Regression Test: ', round(log_reg_score_teste.mean() * 100, 2).astype(str) + '%')

In [ ]:
Y_pred_logreg = logreg.predict(X_test)
cm = confusion_matrix(y_test,Y_pred_logreg)
sns.heatmap(cm,annot=True,fmt="d")

In [ ]:
#KNN. parameters have been choosing using GridSearchCV
knn = KNeighborsClassifier(weights='uniform',n_neighbors=27,leaf_size=6)
knn.fit(X_train,y_train)

In [ ]:
knears_score = cross_val_score(knn, X_train, y_train, cv=10,scoring='roc_auc_ovo')
knears_score_teste = cross_val_score(knn, X_test, y_test, cv=10,scoring='roc_auc_ovo')
print('Score KNN Training: ', round(knears_score.mean() * 100, 2).astype(str) + '%')
print('Score KNN Test: ', round(knears_score_teste.mean() * 100, 2).astype(str) + '%')

In [ ]:
Y_pred_knn = knn.predict(X_test)
cm = confusion_matrix(y_test,Y_pred_knn)
sns.heatmap(cm,annot=True,fmt="d")

In [ ]:
#RANDOM Forest
random_forest = RandomForestClassifier(max_depth=10,n_estimators=350)
random_forest.fit(X_train,y_train)

forest_score = cross_val_score(random_forest, X_train, y_train, cv=10,scoring='roc_auc_ovo')
forest_score_teste = cross_val_score(random_forest, X_test, y_test, cv=10,scoring='roc_auc_ovo')
print('Score RFC Training: ', round(forest_score.mean() * 100, 2).astype(str) + '%')
print('Score RFC Test: ', round(forest_score_teste.mean() * 100, 2).astype(str) + '%')


Y_pred_rf = random_forest.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test,Y_pred_rf)
sns.heatmap(cm,annot=True,fmt="d")

In [ ]:
#Gradient Boosting
grad_boost = GradientBoostingClassifier(n_estimators=65,max_depth=4)
grad_boost.fit(X_train, y_train)

grad_score = cross_val_score(grad_boost, X_train, y_train, cv=10,scoring='roc_auc_ovo')
grad_score_teste = cross_val_score(grad_boost, X_test, y_test, cv=10,scoring='roc_auc_ovo')
print('Score GradBoost Training: ', round(grad_score.mean() * 100, 2).astype(str) + '%')
print('Score GradBoost Test: ', round(grad_score_teste.mean() * 100, 2).astype(str) + '%')

Y_pred_gb = grad_boost.predict(X_test)

In [ ]:
cm = confusion_matrix(y_test,Y_pred_gb)
sns.heatmap(cm,annot=True,fmt="d")